<a href="https://colab.research.google.com/github/ssudhanshu488/Cat-Dog/blob/main/Dog%26Cat_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np

In [2]:
ROWS = 64
COLS = 64
CHANNELS = 3

In [3]:
!unzip Train_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train_data/cat.190.jpg  
  inflating: Train_data/cat.1900.jpg  
  inflating: Train_data/cat.1901.jpg  
  inflating: Train_data/cat.1902.jpg  
  inflating: Train_data/cat.1903.jpg  
  inflating: Train_data/cat.1904.jpg  
  inflating: Train_data/cat.1905.jpg  
  inflating: Train_data/cat.1906.jpg  
  inflating: Train_data/cat.1907.jpg  
  inflating: Train_data/cat.1908.jpg  
  inflating: Train_data/cat.1909.jpg  
  inflating: Train_data/cat.191.jpg  
  inflating: Train_data/cat.1910.jpg  
  inflating: Train_data/cat.1911.jpg  
  inflating: Train_data/cat.1912.jpg  
  inflating: Train_data/cat.1913.jpg  
  inflating: Train_data/cat.1914.jpg  
  inflating: Train_data/cat.1915.jpg  
  inflating: Train_data/cat.1916.jpg  
  inflating: Train_data/cat.1917.jpg  
  inflating: Train_data/cat.1918.jpg  
  inflating: Train_data/cat.1919.jpg  
  inflating: Train_data/cat.192.jpg  
  inflating: Train_data/cat.1920.jpg  
  inflating: Tra

In [4]:
!pip install gdown

In [10]:
import gdown
gdown.download("https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW", "dataset.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW
From (redirected): https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW&confirm=t&uuid=b27e18dc-2d81-4554-b376-88d06e0e2c92
To: /content/dataset.zip
100%|██████████| 136M/136M [00:03<00:00, 39.7MB/s]


'dataset.zip'

In [11]:
!unzip dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train_data/cat.190.jpg  
  inflating: Train_data/cat.1900.jpg  
  inflating: Train_data/cat.1901.jpg  
  inflating: Train_data/cat.1902.jpg  
  inflating: Train_data/cat.1903.jpg  
  inflating: Train_data/cat.1904.jpg  
  inflating: Train_data/cat.1905.jpg  
  inflating: Train_data/cat.1906.jpg  
  inflating: Train_data/cat.1907.jpg  
  inflating: Train_data/cat.1908.jpg  
  inflating: Train_data/cat.1909.jpg  
  inflating: Train_data/cat.191.jpg  
  inflating: Train_data/cat.1910.jpg  
  inflating: Train_data/cat.1911.jpg  
  inflating: Train_data/cat.1912.jpg  
  inflating: Train_data/cat.1913.jpg  
  inflating: Train_data/cat.1914.jpg  
  inflating: Train_data/cat.1915.jpg  
  inflating: Train_data/cat.1916.jpg  
  inflating: Train_data/cat.1917.jpg  
  inflating: Train_data/cat.1918.jpg  
  inflating: Train_data/cat.1919.jpg  
  inflating: Train_data/cat.192.jpg  
  inflating: Train_data/cat.1920.jpg  
  inflating: Tra

In [12]:
!unzip Test_data.zip

Archive:  Test_data.zip
  inflating: Test_data/cat.12000.jpg  
  inflating: Test_data/cat.12001.jpg  
  inflating: Test_data/cat.12002.jpg  
  inflating: Test_data/cat.12003.jpg  
  inflating: Test_data/cat.12004.jpg  
  inflating: Test_data/cat.12005.jpg  
  inflating: Test_data/cat.12006.jpg  
  inflating: Test_data/cat.12007.jpg  
  inflating: Test_data/cat.12008.jpg  
  inflating: Test_data/cat.12009.jpg  
  inflating: Test_data/cat.12010.jpg  
  inflating: Test_data/cat.12011.jpg  
  inflating: Test_data/cat.12012.jpg  
  inflating: Test_data/cat.12013.jpg  
  inflating: Test_data/cat.12014.jpg  
  inflating: Test_data/cat.12015.jpg  
  inflating: Test_data/cat.12016.jpg  
  inflating: Test_data/cat.12017.jpg  
  inflating: Test_data/cat.12018.jpg  
  inflating: Test_data/cat.12019.jpg  
  inflating: Test_data/cat.12020.jpg  
  inflating: Test_data/cat.12021.jpg  
  inflating: Test_data/cat.12022.jpg  
  inflating: Test_data/cat.12023.jpg  
  inflating: Test_data/cat.12024.jpg  
 

In [13]:
TRAIN_DIR = 'Train_data/'
TEST_DIR = 'Test_data/'

In [14]:
train_images = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]
test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

In [15]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [16]:
def prepare_data(images):
    m = len(images)
    X = np.zeros((m, ROWS, COLS, CHANNELS), dtype=np.uint8)
    y = np.zeros((1, m))
    for i, image_file in enumerate(images):
        X[i, :] = read_image(image_file)
        if 'dog' in image_file.lower():
            y[0, i] = 1
        elif 'cat' in image_file.lower():
            y[0, i] = 0
    return X, y

In [17]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [18]:
def propagate(w, b, X, Y):
    m = X.shape[1]

    # FORWARD PROPAGATION (FROM X TO COST)
    z = np.dot(w.T, X) + b  # tag 1
    A = sigmoid(z)  # tag 2
    cost = (-np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))) / m  # tag 5

    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (np.dot(X, (A - Y).T)) / m  # tag 6
    db = np.average(A - Y)  # tag 7

    cost = np.squeeze(cost)
    grads = {"dw": dw,
             "db": db}

    return grads, cost

In [19]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []
    for i in range(num_iterations):
        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)

        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]

        # update w and b
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the costs
        if i % 100 == 0:
            costs.append(cost)

        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    # update w and b to dictionary
    params = {"w": w,
              "b": b}

    # update derivatives to dictionary
    grads = {"dw": dw,
             "db": db}

    return params, grads, costs

In [20]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)

    z = np.dot(w.T, X) + b
    A = sigmoid(z)

    for i in range(A.shape[1]):
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[0, i] > 0.5:
            Y_prediction[[0], [i]] = 1
        else:
            Y_prediction[[0], [i]] = 0

    return Y_prediction

In [21]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0
    return w, b

In [22]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    # initialize parameters with zeros
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]

    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    dict = {"costs": costs,
            "Y_prediction_test": Y_prediction_test,
            "Y_prediction_train": Y_prediction_train,
            "w": w,
            "b": b,
            "learning_rate": learning_rate,
            "num_iterations:": num_iterations}

    return dict

In [23]:
train_set_x, train_set_y = prepare_data(train_images)
test_set_x, test_set_y = prepare_data(test_images)

train_set_x_flatten = train_set_x.reshape(train_set_x.shape[0], ROWS * COLS * CHANNELS).T
test_set_x_flatten = test_set_x.reshape(test_set_x.shape[0], -1).T

train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

In [24]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations=6000, learning_rate=0.003, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.671626
Cost after iteration 200: 0.663768
Cost after iteration 300: 0.658534
Cost after iteration 400: 0.654486
Cost after iteration 500: 0.651100
Cost after iteration 600: 0.648129
Cost after iteration 700: 0.645438
Cost after iteration 800: 0.642949
Cost after iteration 900: 0.640613
Cost after iteration 1000: 0.638401
Cost after iteration 1100: 0.636290
Cost after iteration 1200: 0.634267
Cost after iteration 1300: 0.632320
Cost after iteration 1400: 0.630441
Cost after iteration 1500: 0.628624
Cost after iteration 1600: 0.626863
Cost after iteration 1700: 0.625154
Cost after iteration 1800: 0.623493
Cost after iteration 1900: 0.621878
Cost after iteration 2000: 0.620305
Cost after iteration 2100: 0.618771
Cost after iteration 2200: 0.617275
Cost after iteration 2300: 0.615814
Cost after iteration 2400: 0.614387
Cost after iteration 2500: 0.612991
Cost after iteration 2600: 0.611626
Cost after iteration 2700: 0.610289
Cost

In [31]:
test_image = "/dog.jpeg"
my_image = read_image(test_image).reshape(1, ROWS * COLS * CHANNELS).T
my_predicted_image = predict(d["w"], d["b"], my_image)
print(np.squeeze(my_predicted_image))

1.0


In [32]:
import gdown
gdown.download("https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW", "dataset.zip", quiet=False)
gdown.download("https://drive.google.com/uc?id=1d8-OBFcV4vocRBYEH6SocYnu_eCNoDUn", "dataset1.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW
From (redirected): https://drive.google.com/uc?id=1j8i7PYNgNE9ACqi5rKvZ70tcTdCdJtWW&confirm=t&uuid=f20c2374-67f8-4e65-806c-f8d1e2ffd515
To: /content/dataset.zip
100%|██████████| 136M/136M [00:00<00:00, 153MB/s]
Downloading...
From: https://drive.google.com/uc?id=1d8-OBFcV4vocRBYEH6SocYnu_eCNoDUn
To: /content/dataset1.zip
100%|██████████| 23.2M/23.2M [00:00<00:00, 49.8MB/s]


'dataset1.zip'